In [12]:
# import the modules
import pandas as pd
import os
import re
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

### Import data for model training

In [37]:
data = pd.read_csv('data/full.csv')
data.head(3)

,nameid,name,book_date,city,holding_location,age,height,weight,race,sex,eyes,hair,bookid
0,7482,JAMES DEYO ROBINSON,3/10/2017 10:05:55 AM,"WAUKEE, IA",PCJ,54,"5' 11""",280,Black,Male,Brown,Black,248038
1,754952,CORDERRO ALTON LAURENCE,4/23/2017 8:57:43 AM,"DES MOINES, IA",Cherokee Mental Health,26,"5' 11""",210,Black,Male,Brown,Black,250464
2,644421,TAYLOR LOUIS HARLAN,12/7/2017 4:00:07 PM,"DES MOINES, IA",PCJ,24,"5' 09""",290,White,Male,Green,Blonde,263711


### Function to convert height to meters

In [38]:
# regex to extract feet and inches from height
regex_feet = re.compile("([0-9]+)\'")
regex_inches = re.compile("([0-9]+)\"")

def extract_numbers(x, regex):
    captures = regex.findall(x)
    if len(captures) > 0:
        try:
            return int(captures[0])
        except:
            return None
    else:
        return None
    
# parse the height string to feet and inches
data['feet'] = data['height'].map(lambda i: extract_numbers(i, regex_feet))
data['inches'] = data['height'].map(lambda i: extract_numbers(i, regex_inches))
#data.loc[data.inches > 12,:]

- Correct anomalies in data
- Convert weight from pounds to kg
- Calculate BMI
- Hot code gender column fro sex column

In [5]:
# found some incorrect inches, assume 63" to 6.3"
data['inches'] = data['inches'].map(lambda i: i / 10 if i > 12 else i)
# convert feet/inches to inches
data['height'] = data.apply(lambda row: row['feet'] * 12 + row['inches'], axis=1)
# covert inches to m
data['height'] = data['height'].map(lambda i: i * 2.54 / 100)
# weight pounds => kg
data['weight'] = data['weight'].map(lambda i: i * 0.453592)
# calculate BMI = weight/height^2
data['bmi'] = data.apply(lambda row: row['weight'] / row['height'] / row['height'], axis = 1)
# create gender (number format of sex)
data['gender'] = data['sex'].map(lambda i: 1 if i == 'Male' else 0)

#data[['nameid','age','height','weight','race','sex','eyes','hair']]

### Remove rows with no images

In [6]:
# remove rows that no face images
data['index'] = data['bookid'].map(lambda i: str(i) +'.jpg')
allimage = os.listdir('data/face/')
data = data.loc[data['index'].isin(allimage),:]

# remove rows with invalid BMI
data = data.loc[~data['bmi'].isnull(), :]

# split train/valid
in_train = np.random.random(size = len(data)) <= 0.8
train = data.loc[in_train,:]
test = data.loc[~in_train,:]

print('train data dimension: {}'.format(str(train.shape)))
print('test data dimension:  {}'.format(str(test.shape)))

# output to csv files
train.to_csv('data/train.csv', index = False)
test.to_csv('data/valid.csv', index = False)

train data dimension: (1254, 18)
test data dimension:  (289, 18)


### Import modules to read images, detect faces and generate arrays.
- Also set directories for the test, processed test, train and processed train datasets

In [18]:
from mtcnn.mtcnn import MTCNN
import cv2
import os

test_dir = 'data/test/single_face/'
train_dir = 'data/face/'
test_processed_dir = 'data/test/test_aligned/'
train_processed_dir = 'data/face_aligned/'

os.listdir(test_dir)

['albert-einstein.jpg',
 'barack-obama.jpg',
 'bill-gates.jpg',
 'emma_watson.jpg',
 'hilary_clinton.jpg',
 'kent_zheng.jpg',
 'Kim Jong-un.jpg',
 'leonardo.jpeg',
 'trump.jpg',
 'xi.jpg']

In [39]:
# Test out the imported libraries above
img = cv2.cvtColor(cv2.imread(test_dir + 'trump.jpg'), cv2.COLOR_BGR2RGB)
detector = MTCNN()
print(detector.detect_faces(img))
box = detector.detect_faces(img)[0]

1/1 [==============================] - 0s 359ms/step
[{'box': [355, 75, 266, 347], 'confidence': 0.9983004927635193, 'keypoints': {'left_eye': (432, 209), 'right_eye': (551, 208), 'nose': (492, 276), 'mouth_left': (445, 346), 'mouth_right': (544, 347)}}]
1/1 [==============================] - 0s 46ms/step


### Write function to crop images, detect face from a picture and extract it

In [40]:
from keras.preprocessing import image
from keras.utils.layer_utils import get_source_inputs
from tensorflow.keras.utils import load_img, img_to_array
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

def crop_img(im,x,y,w,h):
    return im[y:(y+h),x:(x+w),:]

def detect_face(face_path):
    img = cv2.cvtColor(cv2.imread(face_path), cv2.COLOR_BGR2RGB)
    box = detector.detect_faces(img)[0]
    return box

def detect_faces(face_path):
    #img = cv2.cvtColor(cv2.imread(face_path), cv2.COLOR_BGR2RGB)
    img = load_img(face_path)
    img = img_to_array(img)
    box = detector.detect_faces(img)
    return box

def draw_box(face_path = './test/trump.jpg', plot = True):
    
    boxes = detect_faces(face_path)
    im = np.array(Image.open(face_path), dtype=np.uint8)
    
    if plot:
        # Create figure and axes
        num_box = len(boxes)
        fig,ax = plt.subplots(1, (1 + num_box))
        fig.set_size_inches(4 * (1 + num_box),4)
        # Display the image
        ax[0].imshow(im)
        ax[0].axis('off')
        # Create a Rectangle patch
        for idx, box in enumerate(boxes):
            box_x, box_y, box_w, box_h = box['box']
            rect = patches.Rectangle((box_x, box_y), box_w, box_h, linewidth=1,edgecolor='r',facecolor='none')
            ax[0].add_patch(rect)
            ax[0].text(box_x, box_y, '{:3.2f}'.format(box['confidence']))
            for i in box['keypoints'].keys():
                circle = patches.Circle(box['keypoints'][i], radius = 5, color = 'red')
                ax[0].add_patch(circle)
            ax[1 + idx].imshow(crop_img(im, *box['box']))
            ax[1 + idx].axis('off')
        plt.show()
    
    res = [crop_img(im, *box['box']) for box in boxes]
    
    return res

### Loop through all the test and train images and extract the faces. Save these faces in the processed folders respectively

In [25]:
#For test folder
from tqdm import tqdm
import shutil
if os.path.exists(test_processed_dir):
    shutil.rmtree(test_processed_dir)
os.mkdir(test_processed_dir)
for img in tqdm(os.listdir(test_dir)):
    box = detect_face(test_dir+img)
    im = plt.imread(test_dir+img)
    cropped = crop_img(im, *box['box'])
    plt.imsave(test_processed_dir+img, crop_img(im, *box['box']))

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 64ms/step


 10%|█         | 1/10 [00:02<00:18,  2.07s/it]

1/1 [==============================] - 0s 52ms/step


 20%|██        | 2/10 [00:03<00:15,  1.88s/it]

...

 80%|████████  | 8/10 [00:25<00:06,  3.50s/it]

1/1 [==============================] - 0s 75ms/step


 90%|█████████ | 9/10 [00:28<00:03,  3.38s/it]

1/1 [==============================] - 0s 57ms/step


100%|██████████| 10/10 [00:31<00:00,  3.19s/it]


In [26]:
def cut_negative_boundary(box):
    res = []
    for x in box['box']:
        if x < 0:
            x = 0
        res.append(x)
    box['box'] = res
    return box

In [27]:
#for train folder
if os.path.exists(train_processed_dir):
    shutil.rmtree(train_processed_dir)
os.mkdir(train_processed_dir)

for img in tqdm(os.listdir(train_dir)):
    try:
        box = detect_face(train_dir+img)
        box = cut_negative_boundary(box)
        im = plt.imread(train_dir+img)
        cropped = crop_img(im, *box['box'])
        plt.imsave(train_processed_dir+img, cropped)
    except:
        print(img)
        continue

  0%|          | 0/1544 [00:00<?, ?it/s]

1/1 [==============================] - 0s 61ms/step


  0%|          | 1/1544 [00:02<1:13:21,  2.85s/it]

1/1 [==============================] - 0s 49ms/step


  0%|          | 2/1544 [00:05<1:07:07,  2.61s/it]

...

1/1 [==============================] - 0s 157ms/step


100%|█████████▉| 1543/1544 [52:47<00:03,  3.22s/it]

1/1 [==============================] - 0s 85ms/step


100%|██████████| 1544/1544 [52:48<00:00,  2.05s/it]

304916.jpg


### Set parameters for model training

In [41]:
mode = 'train' #'train' or 'predict'
model_type = 'vgg16'
model_tag = 'base'
model_id = '{:s}_{:s}'.format(model_type, model_tag)
model_dir = './saved_model/model_{:s}.h5'.format(model_id)
bs = 8
epochs = 2
freeze_backbone = True # True => transfer learning; False => train from scratch

### Import remaining necessary modules
- Import modules from keras
- Import the FacePrediction model

In [42]:
import json
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from pathlib import Path
from Scripts.models import FacePrediction
import glob


allimages = os.listdir('./data/face_aligned/')
train = pd.read_csv('./data/train.csv')
valid = pd.read_csv('./data/valid.csv')

train = train.loc[train['index'].isin(allimages)]
valid = valid.loc[valid['index'].isin(allimages)]

In [16]:
#valid

In [43]:
# create metrics, model dirs
Path('./metrics').mkdir(parents = True, exist_ok = True)
Path('./saved_model').mkdir(parents = True, exist_ok = True)

### Model Training

In [6]:
es = EarlyStopping(patience=3)
ckp = ModelCheckpoint(model_dir, save_best_only=True, save_weights_only=True, verbose=1)
tb = TensorBoard('./tb/%s'%(model_id))
callbacks = [es, ckp]

model = FacePrediction(img_dir = './data/face_aligned/', model_type = model_type)
model.define_model(freeze_backbone = freeze_backbone)
model.model.summary()
if mode == 'train':
    model_history = model.train(train, valid, bs = bs, epochs = epochs, callbacks = callbacks)
else:
    model.load_weights(model_dir)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_1 (Conv2D)               (None, 224, 224, 64  1792        ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 conv1_2 (Conv2D)               (None, 224, 224, 64  36928       ['conv1_1[0][0]']                
                                )                                                           

 sex_bn2 (BatchNormalization)   (None, 128)          512         ['sex_fc2[0][0]']                
                                                                                                  
 bmi_act2 (Activation)          (None, 128)          0           ['bmi_bn2[0][0]']                
                                                                                                  
 age_act2 (Activation)          (None, 128)          0           ['age_bn2[0][0]']                
                                                                                                  
 sex_act2 (Activation)          (None, 128)          0           ['sex_bn2[0][0]']                
                                                                                                  
 dropout_1 (Dropout)            (None, 128)          0           ['bmi_act2[0][0]']               
                                                                                                  
 dropout_3

In [43]:
valid['gender'] = valid.sex.map(lambda i: 1 if i == 'Male' else 0)

In [44]:
metrics = model.evaulate(valid)
metrics['model'] = model_type
with open('./metrics/{:s}.json'.format(model_type), 'w') as f:
    json.dump(metrics, f)

metrics = []
for i in glob.glob('./metrics/*.json'):
    with open(i, 'r') as f:
        res = json.load(f)
    metrics.append(res)
metrics = pd.DataFrame(metrics)
metrics['model'] = metrics['model'].apply(lambda i: '* ' + i if i == model_id else i)
metrics.set_index('model').round(3)

10/10 [==============================] - 1052s 104s/step


,bmi_mae,bmi_cor,age_mae,sex_auc
model,,,,
vgg16,5.576,0.07,7.339,0.998


In [44]:
preds = model.predict('./data/test/test_aligned/albert-einstein.jpg', show_img = True)

1/1 [==============================] - 2s 2s/step


In [45]:
preds

[array([[17.67527]], dtype=float32),
 array([[51.30267]], dtype=float32),
 array([[0.6158781]], dtype=float32)]

### Code to get image from user webcam

In [22]:
# program to capture single image from webcam in python

# importing OpenCV library
import cv2 as cv

# initialize the camera
# If you have multiple camera connected with
# current device, assign a value in cam_port
# variable according to that
cam_port = 0
cam = cv.VideoCapture(cam_port)

# reading the input using the camera
result, image = cam.read()

# If image will detected without any error,
# show result
if result:
    cv.namedWindow("cam-test")
    cv.imshow("cam-test",image)
    #cv.waitKey(0)
    cv.destroyWindow("cam-test")
    cv.imwrite("passport.jpg",image)
    cam.release() 
    # showing result, it take frame name and image
    # output
    #cv.imshow("passport", image)
    
    #convert image to an array
    detector = MTCNN()
    img = cv2.cvtColor(cv2.imread("passport.jpg"), cv2.COLOR_BGR2RGB)
    #detect face from image
    box = detector.detect_faces(img)[0]
    im = plt.imread(str("passport.jpg"))
    #crop image to save only the face area
    cropped = crop_img(im, *box['box'])
    #save the cropped area in a new folder
    plt.imsave('data/processed/face.jpg', cropped)
    # saving image in local storage
    #cv.imwrite("passport.jpg", image)

    # If keyboard interrupt occurs, destroy image
    # window
    #cv.waitKey(0)
    #destroy image window after capture
    #cv.destroyWindow("passport")

    # If captured image is corrupted, moving to else part
else:
    print("No image detected. Please! try again")



1/1 [==============================] - 0s 267ms/step


### Make prediction with the processed image

In [27]:
predictin = model.predict('data/processed/face.jpg')[0][0][0]

1/1 [==============================] - 0s 255ms/step


In [36]:
print(f"Your predicted BMI is {round(float(predictin), 3)}Kg/M")

Your predicted BMI is 17.402Kg/M
